# Bib for external:

In [141]:
import os
import pandas as pd
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
import sys
from dateutil.parser import parse

def find_exponents(df,
                   fractionToAnalyze=1,
                   outputPath='outputs',
                   outputTable=True,
                   outputPlots=True,
                   timescaling='mon', 
                   outputTablename='Testtable',
                   logToScreen=True,
                   columnFillThreshold=0.5,
                   exp_treshold=0.01,
                   r_treshold=0.9,
                   maxrows=5000):
    '''
    find and plot plot exponential data in a pandas dataframe

    :param dataframe df: The dataframe containing date-like and metric columns
    :param str fractionToAnalyze: The last xx percent (0.0-1.0) of data (indexed by the found date columns)
    :param str outputPath: The subfolder to write output table and plots to (created if not existing)
    :param bln outputTable: Write table to outputPath/table (True|False)
    :param bln outputPlots: Plot outputs to outputPath/plots (True|False)
    :param str timescaling: (sec|min|hour|day|mon|year) scaling the exponential factors to the given time unit
    :param str outputTablename: The table name used in the table, and the output plots
    :param bln logToScreen: Print out messages (True|False)
    :param dbl columnFillThreshold: filter out columns with more than xx percent (0.0-1.0) of the data missing
    :param dbl exp_treshold: only show and plot results with an exponent higher than x
    :param dbl r_treshold: only show and plot results with an R squared lower than x
    :param int maxrows: specify the maximum number of rows. If df.rowsize > maxrows a random sample of rows is used
    :return: A Pandas dataframe with a list of all date and metric column combinations with their exponential factor and R²
    :rtype: dataframe
    '''
    
    ############## To Do ###################
    # - Nina: Medium Blogpost and Statistics Paper (something like: https://www.dlr.de/sc/Portaldata/15/Resources/dokumente/pyhpc2011/submissions/pyhpc2011_submission_9.pdf)
    # - Mathias: Check with BMW for Open Source release
    # - Mathias: Create Mailing list for pip publishing
    # - Sebastian: The Exp values seem quite high. Do you know if this powerfit library gives back percentages or absolute values?
    # - Sebastian: Check if R (out of powerfit) is a good measure. shouldn't we use RMSE instead? you can search for ###TODO:sebastian:checkRMSE
    # - Sebastian: Improve error handling
    # - Sebastian: Test on publicly available data if we get errors and if error handling works as intended. 
    #              The goal is that errors don't stop the code execution, because we run it sequentially
    # - Sebastian: Publishing to Github
    # - Sebastian: Publishing to pip
    # - Sebastian: Habe hier noch einige Warnings, wo ich nicht weiß wo die herkommen:
    # C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
    # return (self.a < x) & (x < self.b)
    # C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
    # return (self.a < x) & (x < self.b)
    # C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
    # cond2 = cond0 & (x <= self.a)
    # C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
    # return (self.a < x) & (x < self.b)
    # C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
    # return (self.a < x) & (x < self.b)
    #######################################
    
    
    ######################## Functions: #########################
    def is_nan(x):
        return (x is np.nan or x != x)
    
    def printL(text):
        if logToScreen:
            print(text)
            
    def normalize(lst):
        minval=min(lst)
        maxval=max(lst)
        return [(float(i)-minval)/(maxval-minval) for i in lst]

    def powerfit(x, y):
        """line fitting on y-log scale"""

        slope, intercept, r, _, _=linregress(x,np.log(y))

        printL('slope: '+str(slope))
        printL('intercept:'+str(intercept))
        printL('r²:'+str(r))
        #k, m = np.polyfit(x, np.log(y), 1)
        return slope,intercept,r

    def cleanfilename(filename,cleanlist):
        text=filename
        for cleanword in cleanlist:
            text = text.replace(cleanword, '-')
        text=text.replace('-----','-')
        text=text.replace('----','-')
        text=text.replace('---','-')
        text=text.replace('--','-')
        text=text.replace('-','-')
        if text!=filename:
            printL("Warning: Provided filename '"+filename+"' is invalid. Set to '"+text+"'")
        return text
    
    def checkAndGetBool(val):
        if isinstance(val, str):
            return val.lower() in ['true', '1', 't', 'y', 'yes']
        if isinstance(val, (int, float)):
            if val>=1:
                return 1
            else:
                return 0

        
    ############################ Input handling: #############################
    data=df
    ##### Input handling: logToScreen ###
    logToScreen=checkAndGetBool(logToScreen)
    #### Input handling: maxrows ##############  
    if maxrows<=0:
        rownum=df.shape[0]
        maxrows=min(rownum,5000)
        printL("Warning: maxrows cannot be <1. Set to " + str(rownum))
        
    ##### Input handling: df ###################
    if df is None:
        printL("Warning: Dataframe is empty. Returning None")
        return None
    if df.shape[0]==0:
        printL("Warning: Dataframe is empty. Returning None")
        return None    
    if df.shape[0]>maxrows:
        data=data.sample(maxrows)
        
    ##### Input handling: fractionToAnalyze ###  
    ##### ensure that the fraction is between 0 and 1:   
    if fractionToAnalyze>1:
        printL("Warning: Fraction to analyze was specified > 1 ! Set to 1" )
        fractionToAnalyze=1
    if fractionToAnalyze<=0:
        printL("Warning: Fraction to analyze was specified <= 0 ! Set to 0.1" )
        fractionToAnalyze=0.1
        
    ##### Input handling: outputPath ###  
    ####ensure that output folders exist:
    if outputPlots:
        
        if not os.path.exists(".\\"+outputPath):
            os.makedirs(".\\"+outputPath)
        if not os.path.exists(".\\"+outputPath+"\\plots"):
            os.makedirs(".\\"+outputPath+"\\plots")
    if outputTable:
        if not os.path.exists(".\\"+outputPath):
            os.makedirs(".\\"+outputPath)
        if not os.path.exists(".\\"+outputPath+"\\table"):
            os.makedirs(".\\"+outputPath+"\\table")

    ##### Input handling: outputTable ###
    outputTable=checkAndGetBool(outputTable)

    ##### Input handling: outputPlots ###
    outputPlots=checkAndGetBool(outputPlots)
    
    ##### Input handling: timescaling ###
    if timescaling in ['sec','s','seconds','se','second']:
        timeScalingDivisor=1000
    elif timescaling in ['min','mi','minutes','minute']:
        timeScalingDivisor=1000*60
    elif timescaling in ['hour','h','ho','hou','hours']:
        timeScalingDivisor=1000*60*60    
    elif timescaling in ['day','d','da','days']:
        timeScalingDivisor=1000*60*60*24
    elif timescaling in ['mon','mo','month','months']:
        timeScalingDivisor=1000*60*60*24*30.4366666666
    elif timescaling in ['year','yea','ye','y','years']:
        timeScalingDivisor=1000*60*60*24*30.4366666666*365
    else:
        printL("Warning: No valid timescaling provided (sec|min|hour|day|mon|year). Set to month.")
    
    ##### Input handling: outputTablename ###
    if outputTablename is None:
        printL("Warning: No valid outputTableName provided. As it is used for file naming, 'testtable' is used.")
        outputTablename="testtable"
        
    ##### Input handling: columnFillThreshold ###
    if columnFillThreshold>1:
        printL("Warning: columnFillThreshold was specified > 1 ! Set to 1" )
        columnFillThreshold=1
    if columnFillThreshold<=0:
        printL("Warning: columnFillThreshold was specified <= 0 ! Set to 0.1" )
        columnFillThreshold=0.1

        
        
    ############################ Code: #############################
      
    
    #### specify the columns of the returntable:
    resulttablecols=['tablename','datecol','valuecol','analyzed_fraction','exponent','r_squared'] 
    
    printL ('######## Starting ' + outputTablename + ' ###### fraction '+str(fractionToAnalyze)+'########')
    printL ('##########################################')
    
    #### specify which column types are regarded as numeric (and get analyzed):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    #### specify which chars to remove in the filenames:
    cleanlist=['(',')',' ',':','[',']','#','/']  
    

    #### initialize an empty data frame with the columns for returning the results:
    df_result = pd.DataFrame(columns=resulttablecols)
    
    #### Loop through all datetimecols:
    for datetimecol in data.columns: 
        df_temp=data.copy()

             
        ### identify date columns and append as converted column (datetime) and converted2 (numeric - for correlation)
        
        if any(df_temp[datetimecol].astype(str).str.len()<4):
            printL("####### The datetime column "+ datetimecol+": has less than 4 chars --> not used as datetime column.")
            continue
        try:
            df_temp['date_converted']=pd.to_datetime(df_temp[datetimecol])
            df_temp['date_converted2']=mdates.date2num(df_temp['date_converted'])
        except:
            printL("####### Datetime column "+ datetimecol+": conversion error!")
            continue
        if any(df_temp['date_converted'] < '1990-01-01'):
            printL("####### The datetime column "+ datetimecol+" had values before 2000-01-01 --> not used as date")
            continue
        if datetimecol!='date_converted':
            printL("####### The datetime column: "+ datetimecol + " is used as datetimecol!")

            
        ### filter columns where only 50% of the rows have values:
        df_temp= df_temp.loc[:, df_temp.isnull().mean() < columnFillThreshold]
        
        ### Only use the last fraction of the data, indexed / determined by the time column at hand:
        maxval=max(df_temp['date_converted2']) #use converted time columns to allow for easy multiplication
        minval=min(df_temp['date_converted2'])
        timepoint_to_cut=(maxval-minval)*(1-fractionToAnalyze)+minval
        df_temp=df_temp[(df_temp['date_converted2'] > timepoint_to_cut)]
 
        ###loop through all numeric cols:
        for numericcol in data.select_dtypes(include=numerics).columns:
            if numericcol==datetimecol or numericcol not in df_temp.columns:
                continue ###skip this column if it is the date col
            try:
                #### zero values will break the algorithm with log calculations. 
                #### Therefore, we move all data points slightly above 0.
                #### 'slightly" is defined as min_max_diff/100000

                min_max_diff=max(df_temp[numericcol])-min(df_temp[numericcol])
                min0=False
                if min(df_temp[numericcol])==0:
                       min0=True
                shiftamount=min_max_diff/100000
                #### Start computing the values:
                printL('##### Computing numeric col: '+ numericcol)
                printL('##### Date col: '+datetimecol)
                cols_to_keep=['date_converted','date_converted2',datetimecol,numericcol]
                df_temp_2=df_temp[cols_to_keep].copy()# this df is used for cleaning and afterwards splitted
                #remove all rows with nan or inf:
                df_temp_2=df_temp_2.replace([np.inf, -np.inf], np.nan)
                df_temp_2=df_temp_2.dropna(subset=[numericcol])
                df_temp_2=df_temp_2.sort_values(['date_converted'],ascending=True)

                #now get minimum val, and move all points if negative:
                minval=min(df_temp_2[numericcol])


                #df_temp_2 is used for the regular plot, df_temp_3 is used for log calculations/plots (values are moved here):
                df_temp_3=df_temp_2
                if minval<=0:
                    df_temp_3[numericcol]=df_temp_3[numericcol]+shiftamount 
                    df_temp_3[numericcol]=df_temp_3[numericcol]-minval
                    printL('! moved by '+str(minval) +' to correct for negative values')


                x=df_temp_3['date_converted2']
                y=df_temp_3[numericcol]
                datetime=df_temp_3['date_converted']#datetime




                #### fit a regression line and get the slope (= exponential factor):
                slope,intercept,r=powerfit(x,y)   ###TODO:sebastian:checkRMSE
                y_lin = slope*x + intercept
                
                r=abs(r)
                slope=slope*timeScalingDivisor
                if r<r_treshold or slope<exp_treshold or is_nan(r) or is_nan(slope):
                    continue

                #append to output table:
                df_result=df_result.append(pd.Series([outputTablename,datetimecol,numericcol,fractionToAnalyze,slope,r],index=resulttablecols),ignore_index=True)
                if outputTable:
                    df_result.to_csv(outputPath+'/table/'+cleanfilename(outputTablename+"-"+str(int(round(fractionToAnalyze*100,0)))+"perc.csv",cleanlist))
                    
                    
                ################################### Plotting #########################################

                if outputPlots:
                    ########### Plotparameters:
                    figsize_plot=(11,5) #inches
                    fontsize_plot=12
                    linestyle='-'#'--'
                    linewidth_plot=2
                    ############################ Plot 1: Logarithmized - Dev ############################
                    fig, ax = plt.subplots(figsize=figsize_plot)
                    plt.plot(datetime, y,'.', alpha=.3,color="#3e5463") #plotting the values

                    ##exp to all y values: 
                    ys_exp = [math.exp(x) for x in y_lin]

                    plt.plot(datetime,ys_exp,linestyle, linewidth=linewidth_plot,color='#F38d00') #plotting the line

                    #plotting additional information:
                    plt.annotate("ex="+str(round(slope,4)), xy=(0.95,0.95),xycoords='axes fraction',
                     fontsize=fontsize_plot)
                    plt.annotate("r²="+str(round(r,4)), xy=(0.95,0.90),xycoords='axes fraction',
                     fontsize=fontsize_plot)
                    plt.title(datetimecol+"~"+numericcol + " (fraction: last "+str(100*fractionToAnalyze)+"%)", fontsize=14)
                    plt.yscale('log')
                    if min0: #only set min y axis if 0 values are existing:
                        x1,x2,y1,y2 = plt.axis()
                        plt.axis((x1,x2,shiftamount/10,y2))
                        #plt.annotate("0 (no log)", xy=(-0.005,shiftamount),xycoords=('axes fraction','data'),fontsize=fontsize_plot,color='red',ha='right')

                    plt.xticks(rotation=90)
                    ax.spines['right'].set_visible(False)
                    ax.spines['top'].set_visible(False)
                    plt.savefig(outputPath+'/plots/'+cleanfilename(outputTablename+'-'+datetimecol+'-'+numericcol,cleanlist)+
                                '-'+str(int(round(fractionToAnalyze*100,0)))+'perc-log.png')
                    plt.close('all')

                ############################ Plot 2: Regular Plot - Dev ############################
                    fig, ax = plt.subplots(figsize=figsize_plot)

                    plt.plot(datetime, y,'.', alpha=.3,color="#3e5463") #plotting the values

                    plt.plot(datetime,ys_exp,linestyle, linewidth=linewidth_plot,color='#F38d00') #plotting the line

                    #plotting additional information:
                    plt.annotate("ex="+str(round(slope,4)), xy=(0.95,0.95),xycoords='axes fraction',
                     fontsize=fontsize_plot)
                    plt.annotate("r²="+str(round(r,4)), xy=(0.95,0.90),xycoords='axes fraction',
                     fontsize=fontsize_plot)
                    plt.title(datetimecol+"~"+numericcol + " (fraction: last "+str(100*fractionToAnalyze)+"%)", fontsize=14)
                    if min0: #only set min y axis if 0 values are existing:
                        x1,x2,y1,y2 = plt.axis()
                        plt.axis((x1,x2,shiftamount/10,y2))
                        #plt.annotate("0", xy=(-0.005,shiftamount),xycoords=('axes fraction','data'),fontsize=fontsize_plot,color='red',ha='right')

                    plt.xticks(rotation=90)
                    ax.spines['right'].set_visible(False)
                    ax.spines['top'].set_visible(False)

                    plt.savefig(outputPath+'/plots/'+cleanfilename(outputTablename+'-'+datetimecol+'-'+numericcol,cleanlist)+
                                '-'+str(int(round(fractionToAnalyze*100,0)))+'perc-nonlog.png')
                    plt.close('all')

            except:
                printL("Unexpected error:" + str(sys.exc_info()[0]))
    if df_result.shape[0]>0: ##only append if there are results:
        ##finally create a combined score for easy sorting:

        #exponent_scaled = normalize(list(df_result['exponent']))

        #r² is already scaled, but we want a higher score for lower r values:
        #r_prepared=1-r #a low r should lead to a higher score
        #it is much more important, that is is a power law (low r²), a high exponent is rather secondary --> weight
        #weight_factor_exponent=100
       
        #sort_score=[(weight_factor_exponent*r_prepared+x)/(weight_factor_exponent+1) for x in exponent_scaled]
        #df_result['sort_score']=sort_score
        df_result=df_result.sort_values(['exponent'],ascending=False)     

    return(df_result)

# Examples

## loading testdata in a dataframe

In [132]:
#sqlDFTest = spark.sql("SELECT * FROM db_prepared_bdas_rsu_local_e2e.bt_tal_main limit 10")
#sqlDF.show(1)
#sqlDFTest = spark.sql("select * from db_source_gwk_local.v_bdgwk_t_gw_antrep_gut where rand() <= 0.0001 distribute by rand() sort by rand() limit 1000")
sqlDFTest=pd.read_csv("db_source_gwk_local.v_bdgwk_t_gw_antrep_gut.csv",sep=";")
sqlDFTest.head(2)


,Unnamed: 0,antrag_id,gutschriftsdatum,line_nr,version,ist_lokale_reparatur,gw_kulanzstufe,gw_kulanzstufe_lokal,gw_kulanzstufe_zentral,mixed_stage,...,vfc_id_fl_fzg,vfc_id_fa,vfc_id_fbed,fb_typ_id_fo,fb_typ_id_fl_ort,fb_typ_id_fl_fzg,fb_typ_id_fa,fb_typ_id_fbed,erfassungsdatum,deltakey_ts
0,0,241391683.0,2016-05-07 00:00:00.0,2.0,2.0,0,7,7.0,7.0,0.0,...,NaN,NaN,NaN,FO_K,NaN,NaN,NaN,NaN,2012-11-16 00:00:00.0,2016-05-07
1,1,191737063.0,2011-09-24 00:00:00.0,3.0,2.0,0,7,NaN,7.0,0.0,...,NaN,NaN,NaN,FO_K,NaN,NaN,NaN,NaN,2012-11-16 00:00:00.0,2011-09-25


## Run the function on the test data:

In [129]:
resulttable=find_exponents(sqlDFTest,
                           fractionToAnalyze=1,
                           outputPath='outputs',
                           outputTable=True,
                           outputPlots=True,
                           outputTablename='Testtable',
                           logToScreen=True,
                           columnFillThreshold=0.5,
                           exp_treshold=0.1,
                           r_treshold=0.7,
                           maxrows=5000)


resulttable


######## Starting Testtable ###### fraction 0.3########
##########################################
####### The datetime column Unnamed: 0: has less than 4 chars --> not used as datetime column.
####### The datetime column antrag_id had values before 2000-01-01 --> not used as date
####### The datetime column: gutschriftsdatum is used as datetimecol!
734138.0
737036.0
736166.6
(1000, 141)
(409, 141)
##### Computing numeric col: Unnamed: 0
##### Date col: gutschriftsdatum
slope: -1.683714421366873e-05
intercept:18.283698926280245
r²:-0.004354587359007977
##### Computing numeric col: antrag_id
##### Date col: gutschriftsdatum
slope: 0.00015468709457285563
intercept:-94.56397184190197
r²:0.9989080339925134
##### Computing numeric col: line_nr
##### Date col: gutschriftsdatum
slope: -0.00014547970901803646
intercept:107.74773863323368
r²:-0.0585687678238649
##### Computing numeric col: version
##### Date col: gutschriftsdatum
slope: -5.593283672200104e-05
intercept:41.767453325985045
r²:-0.

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:60: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)



slope: -0.00018277385435147705
intercept:132.0983679046019
r²:-0.01017944270185537
##### Computing numeric col: ag_kost_o_nk_eur
##### Date col: gutschriftsdatum
! moved by -4.71 to correct for negative values
slope: -0.00010564921260426873
intercept:80.92053729493287
r²:-0.016008895726879555
##### Computing numeric col: ag_kost_aw_p_az_eur
##### Date col: gutschriftsdatum
! moved by 0.0 to correct for negative values
slope: -0.00015134643719187543
intercept:111.01609584496597
r²:-0.009164191744126363
##### Computing numeric col: ag_kost_aw_p_az_ust_eur
##### Date col: gutschriftsdatum
! moved by 0.0 to correct for negative values
slope: -0.0005260848220185409
intercept:379.69689104025827
r²:-0.09174922413282548
##### Computing numeric col: ag_kost_aw_p_az_netto_eur
##### Date col: gutschriftsdatum
! moved by 0.0 to correct for negative values
slope: -0.00014436755587237545
intercept:105.87244986809951
r²:-0.008748254206803677
##### Computing numeric col: ag_teilekost_m_nk_eur
##### D

! moved by 0.0 to correct for negative values
slope: -3.891945248932769e-05
intercept:23.822709147814248
r²:-0.008031575360926739
##### Computing numeric col: ag_lc_kost_keine_ust_lw
##### Date col: gutschriftsdatum
! moved by 0.0 to correct for negative values
slope: -5.849871040695097e-07
intercept:1.5887993041009016
r²:-0.00010118502458218633
##### Computing numeric col: ag_fremdl_kost_lw
##### Date col: gutschriftsdatum
! moved by -4.99 to correct for negative values
slope: -0.00012778765633709817
intercept:96.38531196706295
r²:-0.022152132685857678
##### Computing numeric col: ag_fremdl_kost_ust_lw
##### Date col: gutschriftsdatum
! moved by 0.0 to correct for negative values
slope: -3.195588782821971e-05
intercept:18.65498312212456
r²:-0.011919130750062741
##### Computing numeric col: ag_fremdl_kost_netto_lw
##### Date col: gutschriftsdatum
! moved by -4.99 to correct for negative values
slope: -0.00012766477937113422
intercept:96.29407197306746
r²:-0.02216395424443128
##### Comp

C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


...

C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)



##### Computing numeric col: accmgr_hc_kost_eur
##### Date col: deltakey_ts
! moved by -22.6 to correct for negative values
slope: -4.671948718693035e-05
intercept:37.691666127106515
r²:-0.01823199083935092
##### Computing numeric col: accmgr_fremdl_kost_eur
##### Date col: deltakey_ts
! moved by -836.53 to correct for negative values
slope: -0.00015139709359004663
intercept:118.2337375020041
r²:-0.06873047588726819
##### Computing numeric col: accmgr_reparaturkost_eur
##### Date col: deltakey_ts
! moved by -836.53 to correct for negative values
slope: -0.000103192779679952
intercept:82.85757453202115
r²:-0.04747483617354373
##### Computing numeric col: accmgr_plop_eur
##### Date col: deltakey_ts
! moved by -104.95 to correct for negative values
slope: 0.00011035106511714986
intercept:-76.51410211776495
r²:0.04818769277771283
##### Computing numeric col: accmgr_kost_o_nk_lw
##### Date col: deltakey_ts
! moved by -170625.0 to correct for negative values
slope: -6.727437070038244e-05
in

intercept:44.490460424430665
r²:-0.044381026495054765


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:60: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:60: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-pac

,tablename,datecol,valuecol,analyzed_fraction,exponent,r_squared
1,Testtable,ladezeitpunkt,antrag_id,0.3,412574.395030,0.979597
2,Testtable,deltakey_ts,antrag_id,0.3,412574.395030,0.979597
0,Testtable,gutschriftsdatum,antrag_id,0.3,406784.983836,0.998908


## Run multiple queries with different parameters but add them up to one result dataframe

In [140]:
firstrow=True
for fraction in [1,0.5,0.25]:
    temptable=find_exponents(sqlDFTest,
                           fractionToAnalyze=fraction,
                           outputPath='outputs',
                           outputTable=False,
                           outputPlots=False,
                           outputTablename='Testtable',
                           logToScreen=False,
                           columnFillThreshold=0.5,
                           exp_treshold=0.6,
                           r_treshold=0.7,
                           maxrows=5000)
    if firstrow:
        resulttable=temptable
        firstrow=False
    else:
        resulttable=resulttable.append(temptable)
        
resulttable
    

False
0


C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-pac

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-pac

C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages

...

C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value enco

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-pac

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-pac

C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:63: RuntimeWarning: divide by zero encountered in log
C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value enco

,tablename,datecol,valuecol,analyzed_fraction,exponent,r_squared
0,Testtable,gutschriftsdatum,antrag_id,1,375100.184833,0.999400
1,Testtable,ladezeitpunkt,antrag_id,1,374501.571608,0.994495
2,Testtable,deltakey_ts,antrag_id,1,374501.571608,0.994495
1,Testtable,ladezeitpunkt,antrag_id,0.5,403706.156264,0.975731
2,Testtable,deltakey_ts,antrag_id,0.5,403706.156264,0.975731
0,Testtable,gutschriftsdatum,antrag_id,0.5,392123.592223,0.999374
0,Testtable,gutschriftsdatum,antrag_id,0.25,411146.922172,0.998433
1,Testtable,ladezeitpunkt,antrag_id,0.25,410639.939745,0.998503
2,Testtable,deltakey_ts,antrag_id,0.25,410639.939745,0.998503
